In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import warnings
import json
import csv
warnings.filterwarnings('ignore')

# Paths
BASE_DIR = Path.cwd().parent.parent.parent.parent
DATA_DIR = BASE_DIR / "oc_mini"

# Add gnn package to path (parent directory)
sys.path.insert(0, str(Path.cwd().parent))

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Configuration

You can easily swap the transformer model and data source by modifying these variables below.

# GNN Baseline 1: Induced Subgraph with Frozen Transformer

**Experiment Goal**: Demonstrate that when a GNN is trained on an induced subgraph of training nodes (with frozen transformer), test nodes with no edges degrade to transformer-only performance.

**Setup**:
- 90% train nodes, 10% test nodes
- Graph contains ONLY edges between training nodes
- Transformer (SciBERT) is frozen
- Only GNN layers are trainable
- Input: Title + Abstract concatenated

**Expected Result**: Test nodes will have identical embeddings from transformer-only vs GNN, since they have no graph connectivity.

---

In [ ]:
# Import GNN modules
from model import TransformerGNN
from graph_utils import (
    create_induced_subgraph,
    analyze_graph_statistics,
    get_node_texts
)
from split_utils import create_node_based_split

# Data paths
edgelist_path = DATA_DIR / "network" / "oc_mini_edgelist.csv"
metadata_path = DATA_DIR / "metadata" / "oc_mini_node_metadata.csv"

In [ ]:
# Load metadata
metadata_df = pd.read_csv(metadata_path)
print(f"Metadata loaded: {len(metadata_df)} entries")

# Create train/test split (90/10 split)
all_node_ids = [str(node_id) for node_id in metadata_df['id'].values]
train_nodes, test_nodes = create_node_based_split(all_node_ids, test_ratio=0.1, seed=42)

print(f"\nTrain nodes: {len(train_nodes)} ({len(train_nodes)/len(all_node_ids)*100:.1f}%)")
print(f"Test nodes: {len(test_nodes)} ({len(test_nodes)/len(all_node_ids)*100:.1f}%)")

metadata_df.head()

In [ ]:
# Create induced subgraph - KEY STEP FOR BASELINE
# This graph will ONLY contain edges between training nodes
# Test nodes will have NO edges (degree = 0)

edge_index, node_to_idx, idx_to_node = create_induced_subgraph(
    edgelist_path,
    train_nodes,
    metadata_df
)

print(f"\nInduced subgraph created:")
print(f"  Nodes in mapping: {len(node_to_idx)}")
print(f"  Edges: {edge_index.shape[1]}")

# Analyze statistics
analyze_graph_statistics(edge_index, train_nodes, node_to_idx, metadata_df)

In [ ]:
# Initialize TransformerGNN model with SciBERT
# Key: Transformer is FROZEN, only GNN layers are trainable

model_name = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TransformerGNN(
    model_name=model_name,
    gnn_type='gcn',           # Use GCN layers
    hidden_dim=768,           # Match SciBERT output
    num_gnn_layers=2,         # 2 GNN layers
    dropout=0.1,
    pooling='cls',
    freeze_transformer=True   # IMPORTANT: Freeze transformer!
).to(device)

print("\nModel Summary:")
print(f"  Transformer: {model_name}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Trainable (GNN only): {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"  Frozen (Transformer): {sum(p.numel() for p in model.parameters() if not p.requires_grad):,}")

In [ ]:
# BASELINE DEMONSTRATION
# Compare transformer-only vs GNN embeddings for test nodes
# NOTE: get_node_texts() concatenates title + abstract for each node

print("="*70)
print("BASELINE DEMONSTRATION: GNN Degradation for Test Nodes")
print("="*70)

# Sample a few test nodes
import random
sample_test_nodes = random.sample(test_nodes, min(5, len(test_nodes)))

model.eval()
with torch.no_grad():
    for node_id in sample_test_nodes:
        # 1. Get transformer-only embedding
        # get_node_texts concatenates title + abstract
        text = get_node_texts([node_id], metadata_df)[0]
        inputs = tokenizer(
            [text],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)
        
        transformer_emb = model.encode_text(inputs['input_ids'], inputs['attention_mask'])
        transformer_emb = transformer_emb.cpu().numpy()[0]
        
        # 2. Get GNN embedding (full pipeline)
        # First get all transformer embeddings (title + abstract for each)
        all_node_ids = sorted(node_to_idx.keys(), key=lambda x: node_to_idx[x])
        all_texts = get_node_texts(all_node_ids, metadata_df)
        
        # Encode all texts in batches
        all_embs = []
        batch_size = 32
        for i in range(0, len(all_texts), batch_size):
            batch_texts = all_texts[i:i+batch_size]
            batch_inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(device)
            batch_embs = model.encode_text(batch_inputs['input_ids'], batch_inputs['attention_mask'])
            all_embs.append(batch_embs)
        
        x = torch.cat(all_embs, dim=0)
        
        # Apply GNN
        gnn_embs = model(x, edge_index.to(device))
        
        # Get embedding for this test node
        node_idx = node_to_idx[node_id]
        gnn_emb = gnn_embs[node_idx].cpu().numpy()
        
        # 3. Compare
        cosine_sim = (transformer_emb * gnn_emb).sum()
        l2_diff = np.sqrt(((transformer_emb - gnn_emb) ** 2).sum())
        
        print(f"\nTest Node {node_id}:")
        print(f"  Cosine similarity: {cosine_sim:.6f} (1.0 = identical)")
        print(f"  L2 difference: {l2_diff:.8f} (0.0 = identical)")
        
        if cosine_sim > 0.999:
            print(f"  ✓ GNN = Transformer (as expected for nodes with no edges)")
        else:
            print(f"  ! Embeddings differ (unexpected)")

print("\n" + "="*70)
print("KEY INSIGHT:")
print("Test nodes have NO edges in the induced subgraph,")
print("so GNN provides NO benefit - it's just the transformer!")
print("="*70)